# Initial Test of Facial Keypoint Detector
## Without PCA and an SVM for each class

First we need to identify the keypoint detector model that we will be using. 

MTCNN - Tensorflow
https://github.com/ipazc/mtcnn
(Appears to be pretrained)
It appears to provide a few facial keypoints. May be useful for patch based approach probably not for direct-point position approach.

Pytorch-Facial-Keypoints - Pytorch
https://github.com/fiyero/pytorch_facial_keypoints
(Not pretrained)
Generates keypoints around face but excludes ears. Can be used for direct-point position approach.

Openface - in Lua?
https://github.com/cmusatyalab/openface
(Might be pretrained)
Not sure if it generates keypoints

Openface - 
https://github.com/TadasBaltrusaitis/OpenFace
(Pretrained models)
High number of facial points. Can be used for direct-point position approach.

Facial-Landmark
https://github.com/huohuotm/Facial-Landmark
(pretrained models)
High number of facial points. Can be used for direct-point position approach.

Face-Recognition
https://github.com/ageitgey/face_recognition
(Is pretrained and comes as a library)
Detects facial points.

## Preprocessing images
TODO ?

## Processing on keypoints
TODO ?

## For the classifier
https://scikit-learn.org/stable/modules/svm.html
Recommend Linear SVC first

# The Plan
I am planning on using Face-Recognition which provides a wrapper around dlib face-keypoint-detector. (If we wanted to we could write our own).

Program will read in image, run face-recog for key-points, and store key-points in a CSV file.

Then using the key-points

# Second approach is an end-to-end solution

Something akin to https://github.com/ZER-0-NE/ML_problems

In [4]:
import face_recognition
image = face_recognition.load_image_file("./data/raw/Test/22q11/image_10008_jpg.jpg")
face_landmarks_list = face_recognition.face_landmarks(image)

print(image.shape)
print(face_landmarks_list)

(600, 800, 3)
[{'chin': [(339, 265), (322, 302), (309, 343), (299, 386), (297, 428), (303, 470), (316, 507), (332, 541), (360, 563), (395, 573), (434, 566), (470, 552), (501, 532), (529, 506), (551, 477), (575, 445), (596, 411)], 'left_eyebrow': [(372, 251), (395, 244), (419, 255), (440, 272), (455, 294)], 'right_eyebrow': [(496, 323), (525, 329), (551, 340), (571, 360), (578, 385)], 'nose_bridge': [(459, 338), (445, 364), (432, 390), (418, 417)], 'nose_tip': [(385, 417), (394, 429), (407, 441), (424, 444), (442, 446)], 'left_eye': [(384, 287), (401, 285), (419, 295), (424, 320), (408, 315), (390, 304)], 'right_eye': [(491, 364), (511, 358), (530, 370), (538, 391), (522, 390), (503, 380)], 'top_lip': [(350, 454), (370, 456), (390, 464), (400, 472), (415, 476), (434, 488), (451, 500), (442, 498), (410, 485), (395, 481), (384, 474), (357, 459)], 'bottom_lip': [(451, 500), (425, 509), (400, 507), (385, 501), (372, 493), (358, 477), (350, 454), (357, 459), (382, 480), (393, 487), (408, 492

In [7]:
from PIL import Image, ImageDraw
import face_recognition

# Load the jpg file into a numpy array
image = face_recognition.load_image_file("./data/raw/Test/Angelman/image_161_jpg.jpg")

# Find all facial features in all the faces in the image
face_landmarks_list = face_recognition.face_landmarks(image)

pil_image = Image.fromarray(image)
for face_landmarks in face_landmarks_list:
    d = ImageDraw.Draw(pil_image, 'RGBA')

    # Make the eyebrows into a nightmare
    d.polygon(face_landmarks['left_eyebrow'], fill=(68, 54, 39, 128))
    d.polygon(face_landmarks['right_eyebrow'], fill=(68, 54, 39, 128))
    d.line(face_landmarks['left_eyebrow'], fill=(68, 54, 39, 150), width=5)
    d.line(face_landmarks['right_eyebrow'], fill=(68, 54, 39, 150), width=5)

    # Gloss the lips
    d.polygon(face_landmarks['top_lip'], fill=(150, 0, 0, 128))
    d.polygon(face_landmarks['bottom_lip'], fill=(150, 0, 0, 128))
    d.line(face_landmarks['top_lip'], fill=(150, 0, 0, 64), width=8)
    d.line(face_landmarks['bottom_lip'], fill=(150, 0, 0, 64), width=8)

    # Sparkle the eyes
    d.polygon(face_landmarks['left_eye'], fill=(255, 255, 255, 30))
    d.polygon(face_landmarks['right_eye'], fill=(255, 255, 255, 30))

    # Apply some eyeliner
    d.line(face_landmarks['left_eye'] + [face_landmarks['left_eye'][0]], fill=(0, 0, 0, 110), width=6)
    d.line(face_landmarks['right_eye'] + [face_landmarks['right_eye'][0]], fill=(0, 0, 0, 110), width=6)

    pil_image.show()

In [9]:
from sklearn.svm import LinearSVC
from sklearn.datasets import make_classification
X, y = make_classification(n_features=4, random_state=0)
clf = LinearSVC(random_state=0, tol=1e-5)
clf.fit(X, y)
print(clf.coef_)
print(clf.intercept_)
print(clf.predict([[0, 0, 0, 0]]))

[[0.08551853 0.39414771 0.49848086 0.37514369]]
[0.28417571]
[1]


# Load image and generate landmarks

In [16]:
import os
import numpy as np
import face_recognition


def load_dataset(setPath):
    X = []
    y = []

    for label in ['22q11','Angelman','Apert','CDL','Down','FragileX','Marfan','Progeria','Sotos','TreacherCollins','Turner','Williams']:
        directory = os.path.join(setPath,label)
        for f in os.listdir(directory):
            path = os.path.join(directory,f)
            
            if os.path.isfile(path) and f != 'desktop.ini':
                
                # Extract landmarks
                image = face_recognition.load_image_file(path)
                face_landmarks_list = face_recognition.face_landmarks(image)

                # Iterate through detected faces
                keypoints = []
                for face in face_landmarks_list:

                    # Get face center
                    # and convert points into list form
                    cX = 0
                    cY = 0
                    count = 0
                    points = []

                    for featureType in face.keys():
                        for point in face[featureType]:
                            points = points + list(point)

                            cY = cY + point[0]
                            cX = cX + point[1]
                            count = count + 1
                    cX = cX / count
                    cY = cY / count

                    # Center all points
                    for i in range(0,len(points)):
                        if i % 2 == 0:
                            points[i] = points[i] - cY
                        else:
                            points[i] = points[i] - cX

                    # Store result
                    keypoints.append(points)

                # Append to dataset
                for facePoints in keypoints:
                    X.append(facePoints)
                    y.append(label)
    
    return np.array(X), np.array(y)

In [19]:
X_test, y_test = load_dataset('./data/raw/Test')
print(X_test.shape)
print(y_test.shape)

(312, 144)
(312,)


In [18]:
X_train, y_train = load_dataset('./data/raw/Train')
print(X_train.shape)
print(y_train.shape)

(1022, 144)
(1022,)
